In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.io import loadmat
from PIL import Image
import math
import pandas as pd
import scipy.cluster.hierarchy as shc
from sklearn.cluster import AgglomerativeClustering
import seaborn as sns

In [ ]:
joint_name = ['HeadF', 'HeadB', 'HeadL', 'SpineF', 'SpineM', 'SpineL', 
            'Offset1', 'Offset2', 'HipL', 'HipR', 'ElbowL', 'ArmL', 
            'ShoulderL', 'ShoulderR', 'ElbowR', 'ArmR', 'KneeR', 
            'KneeL', 'ShinL', 'ShinR']

joints_idx = [[1, 2], [2, 3], [1, 3], [2, 4], [1, 4], [3, 4], [4, 5], 
            [5, 6], [4, 7], [7, 8], [5, 8], [5, 7], [6, 8], [6, 9], 
            [6, 10], [11, 12], [4, 13], [4, 14], [11, 13], [12, 13], 
            [14, 15], [14, 16], [15, 16], [9, 18], [10, 17], [18, 19], 
            [17, 20]]

In [ ]:
# Loading mat file and returning file
def loadMatFile(fileName):
    mat = loadmat('trainTestSplit.mat')[fileName]
    return mat

In [ ]:
# Plot 3D points given file and frame number
def plotKnownOrder(matFile, numb):
    mat = matFile[numb]
    x = mat[0]
    y = mat[1]
    z = mat[2]

    # loading plot
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')

    # adding points, labels and lines
    try:
        ax.scatter(x,y,z, color='#ff5e5e', s =10, marker='x')
    except:
        pass
    # addLabels(ax, x, y, z)
    drawLines(ax, x, y, z)
    drawAllLines(ax, x, y, z)

    # Labeling plot
    ax.set_title("Rat positioning")
    ax.set_xlabel("X axis")
    ax.set_ylabel("Y axis")
    ax.set_zlabel("Z axis")
    plt.show()

# Draw major lines in the rat model
def drawLines(ax,x, y, z):
    # run through all the connections to draw the points
    for i in range(len(joints_idx)):
        try: 
            # Getting both points to draw line
            idx = joints_idx[i]
            x_line = [x[idx[0]-1], x[idx[1]-1]]
            y_line = [y[idx[0]-1], y[idx[1]-1]]
            z_line = [z[idx[0]-1], z[idx[1]-1]]
            z_coord_1 = x[idx[0]-1], y[idx[0]-1], z[idx[0]-1]
            z_coord_2 = x[idx[1]-1], y[idx[1]-1], z[idx[1]-1]

            # Draw lines
            if i < 3: 
                ax.plot(x_line, y_line, z_line, c="#064ea1", linewidth=4)
            elif i < 6:
                ax.plot(x_line, y_line, z_line, c="#64ccd1", linewidth=4)
            else:
                ax.plot(x_line, y_line, z_line, c="#46b8a7", linewidth=4)
        except:
            pass

# Draw all line from every point 
def drawAllLines(ax,x, y, z):
    joint_len = len(joint_name)
    all_lines = []
    distance = []

    for i in range(joint_len):
        point_dist = []
        for j in range(joint_len):
            point_dist.append(measureDistance(x[i], y[i], z[i], x[j], y[j], z[j]))
            if (not([i,j] in all_lines)):
                x_line = [x[i], x[j]]
                y_line = [y[i], y[j]]
                z_line = [z[i], z[j]]
                ax.plot(x_line, y_line, z_line, color='#b1d8fc', linewidth=0.5)
                all_lines.append([i,j])
                all_lines.append([j,i])
        distance.append(point_dist)
    return distance

In [ ]:
# Get the distance to every single point 
def getAllDistances(matFile, numb):
    joint_len = len(joint_name)
    mat = matFile[numb]
    x = mat[0]
    y = mat[1]
    z = mat[2]
    distance = []

    for i in range(joint_len):
        point_dist = []
        for j in range(joint_len):
            point_dist.append(measureDistance(x[i], y[i], z[i], x[j], y[j], z[j]))
        distance.append(point_dist)

    # normalizes distance data
    distance = normalize(distance)

    ax = heatMap(distance)
    ax.set_title("Distance Matrix")
    ax.set_xlabel("Distance from POI to other points")
    ax.set_ylabel("Point of interest (POI)")

    sort = sortMatrix(distance)
    ax = heatMap(sort)
    ax.set_title("Sorted Distance Matrix")
    ax.set_xlabel("Distance from POI to other points")
    ax.set_ylabel("Point of interest (POI)")
    return distance

# Gets the distance in a 3D coordinate system of 2 points
def measureDistance(x_1, y_1, z_1, x_2, y_2, z_2):
    x = (x_1 - x_2)
    y = (y_1 - y_2)
    z = (z_1 - z_2)
    return float(np.sqrt(x**2 + y**2 + z**2))

# Displays a heat map of inputed array
def heatMap(array):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    plt.imshow(array, cmap='YlOrRd') 
    plt.colorbar()
    plt.grid(False)
    return ax

# Sorts matrix from lowest to highest
def sortMatrix(materix):
    sort = []
    for i in range(len(materix)):
        mat = np.sort(materix[i])
        sort.append(mat)
    return sort

# normalizes a matrix
def normalize(matrix):
    norm = []
    max_numb = max(np.array(matrix).flatten())
    for i in matrix:
        norm.append([float(j)/max_numb for j in i])
    return norm

In [ ]:
# Get the angle to every single point 
def getAllAngles(matFile, numb):
    joint_len = len(joint_name)
    mat = matFile[numb]
    x = mat[0]
    y = mat[1]
    z = mat[2]
    distance = []

    for i in range(joint_len):
        point_dist = []
        for j in range(joint_len):
            point_dist.append(measureDistance(x[i], y[i], z[i], x[j], y[j], z[j]))
        distance.append(point_dist)

    # normalizes distance data
    distance = normalize(distance)

    ax = heatMap(distance)
    ax.set_title("Distance Matrix")
    ax.set_xlabel("Distance from POI to other points")
    ax.set_ylabel("Point of interest (POI)")

    sort = sortMatrix(distance)
    ax = heatMap(sort)
    ax.set_title("Sorted Distance Matrix")
    ax.set_xlabel("Distance from POI to other points")
    ax.set_ylabel("Point of interest (POI)")
    return distance

In [ ]:
mat = loadMatFile('split1True')

In [ ]:
for i in range (1):
    # plotKnownOrder(mat, i)
    getAllDistances(mat, i)  
    # getAllAngles(mat, i)

In [ ]:
def getTrainingLabels(numb):
    output = []
    for i in range(numb):
        for j in range(len(joint_name)):
            output.append(j)
    output = np.array(output)
    return output